In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
nursery = fetch_ucirepo(id=76) 
  
# data (as pandas dataframes) 
X = nursery.data.features 
y = nursery.data.targets 
  
# variable information 
print(nursery.variables) 


       name     role         type demographic  \
0   parents  Feature  Categorical        None   
1  has_nurs  Feature  Categorical        None   
2      form  Feature  Categorical        None   
3  children  Feature  Categorical        None   
4   housing  Feature  Categorical        None   
5   finance  Feature  Categorical        None   
6    social  Feature  Categorical        None   
7    health  Feature  Categorical        None   
8     class   Target  Categorical        None   

                                         description units missing_values  
0                     usual, pretentious, great_pret  None             no  
1  proper, less_proper, improper, critical, very_...  None             no  
2            complete, completed, incomplete, foster  None             no  
3                                      1, 2, 3, more  None             no  
4                    convenient, less_conv, critical  None             no  
5                                 convenient, inconv 